<a href="https://colab.research.google.com/github/jongkwonwon/LGE_DX3/blob/main/75QNED90_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# roboflow data 사용(Dataset에 맞게 링크 수정)
# 속도 증가를 위해 런타임 → 런타임 유형 변경 → GPU 설정
!curl -L -o robo.zip https://app.roboflow.com/ds/Da90ZSHMbG?key=ST3dGjRL8j

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1524      0 --:--:-- --:--:-- --:--:--  1527
100 10.5M  100 10.5M    0     0  5320k      0  0:00:02  0:00:02 --:--:-- 8758k


In [2]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [3]:
!mv robo.zip ./dataset

In [4]:
%cd dataset
!unzip -oq robo.zip

/content/dataset


In [5]:
# Yolov5 환경설정
# 초기화

%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [6]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5


In [7]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 16
names: ['NG1', 'NG2', 'NG3', 'NG4', 'NG5', 'NG6', 'NG7', 'NG8', 'OK1', 'OK2', 'OK3', 'OK4', 'OK5', 'OK6', 'OK7', 'OK8']

roboflow:
  workspace: lg-akmpe
  project: 75qned90_test_1
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/lg-akmpe/75qned90_test_1/dataset/1

In [8]:
%cd /
from glob import glob
train_img_list = glob('/content/dataset/train/images/*.jpg')
val_img_list = glob('/content/dataset/valid/images/*.jpg')
print(len(train_img_list)), print(len(val_img_list))

/
96
12


(None, None)

In [9]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [10]:
#yaml 파일 업데이트

import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 16, 'names': ['NG1', 'NG2', 'NG3', 'NG4', 'NG5', 'NG6', 'NG7', 'NG8', 'OK1', 'OK2', 'OK3', 'OK4', 'OK5', 'OK6', 'OK7', 'OK8'], 'roboflow': {'workspace': 'lg-akmpe', 'project': '75qned90_test_1', 'version': 1, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/lg-akmpe/75qned90_test_1/dataset/1'}}


In [11]:
data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'test': '../test/images', 'nc': 16, 'names': ['NG1', 'NG2', 'NG3', 'NG4', 'NG5', 'NG6', 'NG7', 'NG8', 'OK1', 'OK2', 'OK3', 'OK4', 'OK5', 'OK6', 'OK7', 'OK8'], 'roboflow': {'workspace': 'lg-akmpe', 'project': '75qned90_test_1', 'version': 1, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/lg-akmpe/75qned90_test_1/dataset/1'}}


In [12]:
%cat /content/dataset/data.yaml

names:
- NG1
- NG2
- NG3
- NG4
- NG5
- NG6
- NG7
- NG8
- OK1
- OK2
- OK3
- OK4
- OK5
- OK6
- OK7
- OK8
nc: 16
roboflow:
  license: CC BY 4.0
  project: 75qned90_test_1
  url: https://universe.roboflow.com/lg-akmpe/75qned90_test_1/dataset/1
  version: 1
  workspace: lg-akmpe
test: ../test/images
train: /content/dataset/train.txt
val: /content/dataset/val.txt


In [13]:
!cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]

In [14]:
# N회 반복학습 실시
# Batch, epochs는 적절히 수정
# name은 생성되는 폴더명
# /content/yolov5/runs/
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 500 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name 32qned65

/content/yolov5
2023-11-10 05:58:58.280965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 05:58:58.281038: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 05:58:58.281094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=Fal